In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#def create_file_reader_ops(filename_queue):
#    reader = tf.TextLineReader(skip_header_lines=1)
#    _, csv_row = reader.read(filename_queue)
#    record_defaults = [[0], [0], [0], [0], [0], [0]]
#    country, code, gold, silver, bronze, total = tf.decode_csv(csv_row, record_defaults=record_defaults)
#    features = tf.pack([gold, silver, bronze])
#    return features, country

In [3]:
def one_hot(label_vec):
    if label_vec == "asian":
       one_hot = 0
    elif label_vec == "country":
       one_hot = 1
    elif label_vec == "pop":
       one_hot = 2
    elif label_vec == "rock":
       one_hot = 3
    elif label_vec == "hip_hop":
       one_hot = 4
    else: print "label not found"
    return one_hot

filenames = ["asian_features.txt"]



In [4]:
csv = pd.read_csv("asian_features.txt", comment = "%").values
with tf.python_io.TFRecordWriter("csv.tfrecords") as tf_writer:
    for row in csv:
       #file formate : feature 1…..feature n, label
       features, label = row[:-1], row[-1]
       example = tf.train.Example()
       example.features.feature["features"].float_list.value.extend(features)
       o_h_labels = one_hot(label)
       example.features.feature["label"].int64_list.value.append(o_h_labels)
       tf_writer.write(example.SerializeToString())

In [5]:
#get the data set somehow
#parameters

learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

#network parameters
n_hidden = 10
n_input = 124
n_classes = 5

#dataset = tf.data.TFRecordDataset("csv.tfrecords")

def read_and_decode_single_example(filename):
    # first construct a queue containing a list of filenames.
    # this lets a user split up there dataset in multiple files to keep
    # size down
    filename_queue = tf.train.string_input_producer([filename],
                                                    num_epochs=None)
    # Unlike the TFRecordWriter, the TFRecordReader is symbolic
    reader = tf.TFRecordReader()
    # One can read a single serialized example from a filename
    # serialized_example is a Tensor of type string.
    _, serialized_example = reader.read(filename_queue)
    # The serialized example is converted back to actual values.
    # One needs to describe the format of the objects to be returned
    features = tf.parse_single_example(
        serialized_example,
        features={
            # We know the length of both fields. If not the
            # tf.VarLenFeature could be used
            'features': tf.FixedLenFeature([124], tf.float32),
            'label': tf.FixedLenFeature([1], tf.int64)
        })
    # now return the converted data
    label = features['label']
    feat = features['features']
    return label, feat


X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden])),
    'w2': tf.Variable(tf.random_normal([n_hidden, n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}

def mlp(x):
   #hidden layer 1
   h_layer = tf.matmul(x, weights['w1'])
   h_layer_2 = tf.matmul(h_layer, weights['w2']) 
   out_layer = tf.matmul(h_layer_2, weights['out'])
   return out_layer

#construct model
logits = mlp(X)

#define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


In [ ]:
# get single examples
label, feat = read_and_decode_single_example("csv.tfrecords")
# groups examples into batches randomly
labels_batch, feat_batch = tf.train.shuffle_batch(
    [label, feat], batch_size=100,
    capacity=2000,
    min_after_dequeue=1000)

init = tf.global_variables_initializer()


In [ ]:
with tf.Session() as sess:
   sess.run(init)

   #training cycle
   train_ac = []
   test_ac = []
   x_axis = np.arange(1,training_epochs+1)
   for epoch in range(training_epochs):
      avg_cost = 0.
      total_batch = 1
      acc = []
      # Loop over all batches
      for i in range(1):
         batch_x, batch_y= sess.run([feat_batch, labels_batch])
         # Run optimization op (backprop) and cost op (to get loss value)
         _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
#         _, c = sess.run([train_op, loss_op], feed_dict={X: labels_batch, Y: feat_batch})
         # Compute average loss
         avg_cost += c / total_batch
         # Display logs per epoch step
        # Display logs per epoch step
     ## if epoch % display_step == 0:
      print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
      # Test model
      pred = tf.nn.softmax(logits)  # Apply softmax to logits
      correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
      # Calculate accuracy
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
      train_ac.append(accuracy.eval({X: feat, Y: labels}))
      test_ac.append(accuracy.eval({X: feat, Y: labels}))

   print("Accuracy of Train", accuracy.eval({X: feat, Y: labels}))
   print("Accuracy of Test", accuracy.eval({X: feat, Y: labels}))
    
   print("Optimization Finished for full MNIST Data Set!")
   plt.plot(x_axis, test_ac)
   plt.xlabel("Number of Epochs")
   plt.ylabel("Test Accuracy")
   plt.show()
   plt.plot(x_axis, train_ac)
   plt.xlabel("Number of Epochs")
   plt.ylabel("Training Accuracy")
   plt.show()
   print ("batch_size = 100")
   print ("learning_rate = 0.001")

In [ ]:
mnist.train._labels = y_train
mnist.train._images = x_train
mnist.train._num_examples = 9999

with tf.Session() as sess:
   sess.run(init)

   #training cycle
   train_ac = []
   test_ac = []
   x_axis = np.arange(1,training_epochs+1)
   for epoch in range(training_epochs):
      avg_cost = 0.
      total_batch = int(mnist.train.num_examples/batch_size)
      acc = []
      # Loop over all batches
      for i in range(total_batch):
         tf.train.start_queue_runners(sess=sess)
         batch_x, batch_y = mnist.train.next_batch(batch_size)
         # Run optimization op (backprop) and cost op (to get loss value)
         _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
         # Compute average loss
         avg_cost += c / total_batch
         # Display logs per epoch step
        # Display logs per epoch step
      if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
      # Test model
      pred = tf.nn.softmax(logits)  # Apply softmax to logits
      correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
      # Calculate accuracy
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
      train_ac.append(accuracy.eval({X: mnist.train.images, Y: mnist.train.labels}))
      test_ac.append(accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

   print("Accuracy of Train", accuracy.eval({X: mnist.train.images, Y: mnist.train.labels}))
   print("Accuracy of Test", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    
   print("Optimization Finished for small dataset!")
   plt.plot(x_axis, test_ac)
   plt.xlabel("Number of Epochs")
   plt.ylabel("Test Accuracy")
   plt.show()
   plt.plot(x_axis, train_ac)
   plt.xlabel("Number of Epochs")
   plt.ylabel("Training Accuracy")
   plt.show()
   print ("batch_size = 100")
   print ("learning_rate = 0.001")